### This notebook is to generate df of Positions, that is resulted in our strategy

In [1]:
import pandas as pd
from math import sqrt
import math

import requests
import json
import numpy as np

import UNI_v3_funcs as liq_amounts
import load_data
import collections

In [2]:
pip3 install pandas

SyntaxError: invalid syntax (3029356636.py, line 1)

### Query Functions

In [3]:
def get_cprice_tick_at_mint(token0, token1, feeTier, mint_time, decimal0, decimal1):
    
    # get token_ids
    token0_id = load_data.get_token_id(token0)
    token1_id = load_data.get_token_id(token1)
    pool_id = load_data.get_pool_id(token0_id, token1_id, feeTier)
        
    # query price at that specific time
    query_ = """
    {{
      poolHourDatas(first:1,
      where:{{ pool: "{}",
      periodStartUnix: {} }}
      ){{
        tick
      }}
    }}""".format(pool_id, mint_time)
    print(query_)
    
    query_result_ = load_data.run_query(query_)
    json_data_ = json.loads(query_result_.text)
    
    ctick = int(json_data_['data']['poolHourDatas'][0]['tick'])
    cprice = tick_to_price(ctick,decimal0, decimal1)
    
    return cprice, ctick

def get_decimals(token_symbol):
    query_ = """
    {{
      tokens(first:10,
      where:{{ symbol: "{}"}}
      ){{
        decimals
      }}
    }}""".format(token_symbol)
    
    query_result_ = load_data.run_query(query_)
    json_data_ = json.loads(query_result_.text)
    decimal_ = float(json_data_['data']['tokens'][0]['decimals'])
    
    return decimal_

In [101]:
tick_to_price(-41563 ,DECIMAL0, DECIMAL1)

1.566866847891531e-14

### Calculations Functions

In [4]:
def price_to_tick(price_, decimal0_, decimal1_):
    # convert price (token1 by token0) to tick
#     tick_ = math.floor(math.log(sqrt(price_), math.sqrt(1.0001)))
    tick_ = math.floor( math.log(price_*10**(decimal1_-decimal0_), 1.0001) )
    return tick_

def tick_to_price(tick_,decimal0_, decimal1_):
    price_ = (1.0001**tick_/10**(decimal1_-decimal0_))
#     price_ = 1.0001**tick_
    return price_

In [5]:
# L_you calculation / inRange
# 4B. JNP: Use 'get_liquidity' function to calculate liquidity as a function of amounts and price range
def get_liquidity0(sqrtA,sqrtB,amount0,decimals):
    
    if (sqrtA > sqrtB):
          (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
    liquidity=amount0/((2**96*(sqrtB-sqrtA)/sqrtB/sqrtA)/10**decimals)
    return liquidity

def get_liquidity1(sqrtA,sqrtB,amount1,decimals):
    
    if (sqrtA > sqrtB):
        (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
    liquidity=amount1/((sqrtB-sqrtA)/2**96/10**decimals)
    return liquidity

def get_liquidity(tick,tickA,tickB,amount0,amount1,decimal0,decimal1):
    
        sqrt=(1.0001**(tick/2)*(2**96))
        sqrtA=(1.0001**(tickA/2)*(2**96))
        sqrtB=(1.0001**(tickB/2)*(2**96))
        if (sqrtA > sqrtB):
            (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
        if sqrt<=sqrtA:
            
            liquidity0=get_liquidity0(sqrtA,sqrtB,amount0,decimal0)
            inRange = False
            return liquidity0, inRange
        elif sqrt<sqrtB and sqrt>sqrtA:

            liquidity0=get_liquidity0(sqrt,sqrtB,amount0,decimal0)
            liquidity1=get_liquidity1(sqrtA,sqrt,amount1,decimal1)
            
            liquidity=liquidity0 if liquidity0<liquidity1 else liquidity1
            inRange = True
            return liquidity, inRange
        
        else:
            liquidity1=get_liquidity1(sqrtA,sqrtB,amount1,decimal1)
            inRange = False
            return liquidity1, inRange


In [6]:
# Calculate Fees 
def get_fees(lower_tick_, upper_tick_, ctick_,  L_Pool_, swap_Volume_, amount0_p, amount1_p, amt0_, amt1_, decimal0_, decimal1_, feeTier, mint_tick):
    # (lower_tick_, upper_tick_) from df_positions
    # (ctick, L_Pool, swap_volume, amount0_p, amount1_p) from df_merged
    # (amt0, amt1) - this changes based on swaps, but doesnt matter, bacause liquidity always constant
    # (decimal0, decimal1, feetier, mint_tick) - fixeds
    
    # Ver. fixed liquidity ------------------------
    mint_liquidity = liq_amounts.get_liquidity(mint_tick, lower_tick_,upper_tick_, amt0_, amt1_, decimal0_, decimal1_)
    N_amt0, N_amt1 = liq_amounts.get_amounts(ctick_, lower_tick_, upper_tick_, mint_liquidity, decimal0_, decimal1_)
    L_you_ = liq_amounts.get_liquidity(ctick_, lower_tick_,upper_tick_, N_amt0, N_amt1, decimal0_, decimal1_)
    
    # redudant, L_you_ always fixed in this case, L_you_ = mint_liquidity
    
    # Ver. varied liquidity ------------------------
    L_you_old, inRange_ = get_liquidity(ctick_, lower_tick_, upper_tick_, amt0_, amt1_, decimal0_, decimal1_)
#     _, inRange_ = get_liquidity(ctick_, lower_tick_,upper_tick_, amt0_, amt1_, decimal0_, decimal1_)
    # -------------------------------
    
    if inRange_:
        pool_share_ = (L_you_/L_Pool_)
        pool_fee_rate_ = float(feeTier) / 10000
        fees_0 = pool_share_ * amount0_p * pool_fee_rate_ / 100
        fees_1 = pool_share_ * amount1_p * pool_fee_rate_ / 100
    else:
        fees_0 = 0
        fees_1 = 0

    return fees_0, fees_1

In [37]:
def get_liquidity(tick,tickA,tickB,amount0,amount1,decimal0,decimal1):
    
        sqrt=(1.0001**(tick/2)*(2**96))
        sqrtA=(1.0001**(tickA/2)*(2**96))
        sqrtB=(1.0001**(tickB/2)*(2**96))
        if (sqrtA > sqrtB):
            (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
        if sqrt<=sqrtA:
            
            liquidity0=get_liquidity0(sqrtA,sqrtB,amount0,decimal0)
            inRange = False
            return liquidity0, inRange
        elif sqrt<sqrtB and sqrt>sqrtA:

            liquidity0=get_liquidity0(sqrt,sqrtB,amount0,decimal0)
            liquidity1=get_liquidity1(sqrtA,sqrt,amount1,decimal1)
            
            liquidity=liquidity0 if liquidity0<liquidity1 else liquidity1
            inRange = True
            return liquidity, inRange
        
        else:
            liquidity1=get_liquidity1(sqrtA,sqrtB,amount1,decimal1)
            inRange = False
            return liquidity1, inRange
        
print(get_liquidity(41351, 69081, 69091, 1000, 1, 0, 0))
print(get_liquidity(41351, 59081, 59091, 1000.5, 1, 0, 0))

(63264507.2265903, False)
(38392008.96045258, False)


In [7]:
# Step 4: use JNP's code
def get_impermanent_loss():
    return IL

In [8]:
# Step 4
def get_PNL():
    return pnl

In [9]:
# Step 5
def get_cost():
    return cost

### Positions Genrations

In [22]:
df_merged

,Unnamed: 0,Unnamed: 0.1,close,feeGrowthGlobal0X128,feeGrowthGlobal1X128,feesUSD,high,liquidity,low,open,...,pool.token0.symbol,pool.token1.symbol,amount0,amount1,amount0_p,amount0_n,amount1_p,amount1_n,amountUSD,swaps_txCount
0,2139,2139,3371.050772,0,127510492160377860903322733723813,0,3371.050772,222633640125805970242,0.000000,0.000000,...,DAI,WETH,-33.854156,1.000000e-02,0.000000,-33.854156,1.000000e-02,0.000000,0.000000,1.0
1,2138,2138,3371.050602,0,127510936560969447327942820076732,0,3371.050602,229713263233648638236,3371.050602,3371.050602,...,DAI,WETH,-0.000336,1.000000e-07,0.000000,-0.000336,1.000000e-07,0.000000,0.000000,1.0
2,2137,2137,3371.050602,0,127510936560969447327942820076732,0,3371.050602,232295550235364197007,3371.050602,3371.050602,...,DAI,WETH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2136,2136,3371.050602,0,127510936560969447327942820076732,0,3371.050602,379427976363351507096,3371.050602,3371.050602,...,DAI,WETH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2135,2135,3270.498157,0,127510936560969447327942820076732,0,3371.050602,379427976363351507096,3270.498157,3371.050602,...,DAI,WETH,-331.043259,1.000000e-01,0.000000,-331.043259,1.000000e-01,0.000000,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2135,4,4,2593.585043,776968668977177358387607138098571791495,309438643325163112370527963213372068,0,2593.585043,3462780666963265022195495,2570.614474,2570.614474,...,DAI,WETH,194033.308783,-7.470017e+01,218425.038876,-24391.730093,9.509439e+00,-84.209613,242804.419226,11.0
2136,3,3,2593.585043,776978292203169980225606163848602905664,309438643325163112370527963213372068,0,2593.585043,3462780666963265022195495,2593.585043,2593.585043,...,DAI,WETH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2137,2,2,2588.330996,777027753133678150409983803188748783394,309490999796475164960964115559844674,0,2591.982090,575350605560595522715979,2580.129794,2591.982090,...,DAI,WETH,-116868.820672,4.533601e+01,46549.975295,-163418.795967,6.329509e+01,-17.959074,210255.215380,13.0
2138,1,1,2583.740595,777208523032394553427914048177218497946,309574227351200615371422895252255042,0,2588.338457,575350605560595522715979,2567.063155,2588.338457,...,DAI,WETH,-25682.970182,1.018240e+01,94960.184604,-120643.154786,4.694395e+01,-36.761544,215901.771622,10.0


### PNL Wrapper code

In [23]:

mint_liquidity = liq_amounts.get_liquidity(mint_tick, lower_tick_,upper_tick_, amt0_, amt1_, decimal0_, decimal1_)

NameError: name 'mint_tick' is not defined

### Main

In [12]:
# STEP 0:  PROVIDE BY USERS
UPPER = 1/1844.6164  # token1 per token0 (in terms of price)
LOWER = 1/2858.3641  # 1/2376.6075 
FEETIER = '3000' # 0.3 = 3000/10000
TOKEN0 = 'USDC'
TOKEN1 = 'WETH'
MINT_TIME = 1622390400 # needs to start with the hour (unixTime % 3600 = 0) 
BURN_TIME = 1627578000 # 1627578000 # 1625936400 # 
# May 30 - July 30 / 11

In [13]:
# Get required constants
DECIMAL0 = get_decimals(TOKEN0)
DECIMAL1 = get_decimals(TOKEN1)
print(DECIMAL0, DECIMAL1)

6.0 18.0


In [14]:
MINT_PRICE, MINT_TICK = get_cprice_tick_at_mint(TOKEN0, TOKEN1, FEETIER, MINT_TIME, DECIMAL0, DECIMAL1)
print(MINT_PRICE, 1/MINT_PRICE)
print(MINT_TICK)

 
get_token_id: USDC
{'data': {'tokens': [{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'name': 'USD Coin', 'symbol': 'USDC'}]}}
 
get_token_id: WETH
{'data': {'tokens': [{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'name': 'Wrapped Ether', 'symbol': 'WETH'}]}}

 get_pool_id for feeTier: 3000
{'data': {'pools': [{'feeTier': '3000', 'id': '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8', 'token0': {'symbol': 'USDC'}, 'token1': {'symbol': 'WETH'}}]}}

    {
      poolHourDatas(first:1,
      where:{ pool: "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8",
      periodStartUnix: 1622390400 }
      ){
        tick
      }
    }
0.0004165877310455854 2400.4547553287744
198486


In [15]:
# STEP 1 (TODO): CALCULATED BY ALGORITHM, JNP's amounts_relation ?
# THIS CHANGES over time when swaps happens, and we need this to calculate IL
MINT_AMT0 = 252208.7053
MINT_AMT1 = 99.0604

In [16]:
# 1.0001**198486

In [17]:
# Get DF_Merged (accumulated swaps data)
df_merged = pd.read_csv('../data/df_merged_tmp_USDC-WETH-3000-timestamp-1627909200-1620158400.csv')
# df_merged = load_data.get_data(TOKEN0, TOKEN1, FEETIER)

In [103]:
df_merged

,Unnamed: 0,Unnamed: 0.1,close,feeGrowthGlobal0X128,feeGrowthGlobal1X128,feesUSD,high,liquidity,low,open,...,pool.token0.symbol,pool.token1.symbol,amount0,amount1,amount0_p,amount0_n,amount1_p,amount1_n,amountUSD,swaps_txCount
0,2139,2139,3371.050772,0,127510492160377860903322733723813,0,3371.050772,222633640125805970242,0.000000,0.000000,...,DAI,WETH,-33.854156,1.000000e-02,0.000000,-33.854156,1.000000e-02,0.000000,0.000000,1.0
1,2138,2138,3371.050602,0,127510936560969447327942820076732,0,3371.050602,229713263233648638236,3371.050602,3371.050602,...,DAI,WETH,-0.000336,1.000000e-07,0.000000,-0.000336,1.000000e-07,0.000000,0.000000,1.0
2,2137,2137,3371.050602,0,127510936560969447327942820076732,0,3371.050602,232295550235364197007,3371.050602,3371.050602,...,DAI,WETH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2136,2136,3371.050602,0,127510936560969447327942820076732,0,3371.050602,379427976363351507096,3371.050602,3371.050602,...,DAI,WETH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2135,2135,3270.498157,0,127510936560969447327942820076732,0,3371.050602,379427976363351507096,3270.498157,3371.050602,...,DAI,WETH,-331.043259,1.000000e-01,0.000000,-331.043259,1.000000e-01,0.000000,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2135,4,4,2593.585043,776968668977177358387607138098571791495,309438643325163112370527963213372068,0,2593.585043,3462780666963265022195495,2570.614474,2570.614474,...,DAI,WETH,194033.308783,-7.470017e+01,218425.038876,-24391.730093,9.509439e+00,-84.209613,242804.419226,11.0
2136,3,3,2593.585043,776978292203169980225606163848602905664,309438643325163112370527963213372068,0,2593.585043,3462780666963265022195495,2593.585043,2593.585043,...,DAI,WETH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2137,2,2,2588.330996,777027753133678150409983803188748783394,309490999796475164960964115559844674,0,2591.982090,575350605560595522715979,2580.129794,2591.982090,...,DAI,WETH,-116868.820672,4.533601e+01,46549.975295,-163418.795967,6.329509e+01,-17.959074,210255.215380,13.0
2138,1,1,2583.740595,777208523032394553427914048177218497946,309574227351200615371422895252255042,0,2588.338457,575350605560595522715979,2567.063155,2588.338457,...,DAI,WETH,-25682.970182,1.018240e+01,94960.184604,-120643.154786,4.694395e+01,-36.761544,215901.771622,10.0


In [65]:
class BlockPosition:
    def __init__(self, starting_tick, block_size, block_spacing, block_amount):
        self.block_size = block_size
        self.block_spacing = block_spacing
        self.block_amount = block_amount
        self.k = (block_amount-1)/2
        self.center_tick = starting_tick
        self.next_pos_index = 1
        self.block_array = self.gen_block_array()
        
    def gen_block_array(self):
        block_array = collections.deque()
        first_tick = int(self.center_tick - self.k * self.block_spacing)
        for i in range(block_amount):
            curr_tick = int(first_tick + i*self.block_spacing)
            block_array.append({ 
                'index': self.next_pos_index,
                'lower_tick': curr_tick - int(self.block_size/2),
                'upper_tick': curr_tick - int(self.block_size/2) + self.block_size - 1
            })
            self.next_pos_index = self.next_pos_index + 1
        return block_array
    
    def shift_left(self):
        self.center_tick -= self.block_spacing
        prev_lower_tick = self.block_array[0]['lower_tick']
        prev_upper_tick = self.block_array[0]['upper_tick']
        
        self.block_array.appendleft({ 
            'index': self.next_pos_index,
            'lower_tick': prev_lower_tick - self.block_spacing,
            'upper_tick': prev_upper_tick - self.block_spacing
        })
        self.block_array.pop()

        self.next_pos_index += 1
 
    def shift_right(self):
        self.center_tick += self.block_spacing
        prev_lower_tick = self.block_array[-1]['lower_tick']
        prev_upper_tick = self.block_array[-1]['upper_tick']
        
        self.block_array.append({ 
            'index': self.next_pos_index,
            'lower_tick': prev_lower_tick + self.block_spacing,
            'upper_tick': prev_upper_tick + self.block_spacing
        })
        self.block_array.popleft()

        self.next_pos_index += 1




df_merged = df_merged.sort_values('periodStartUnix', ignore_index = True)


In [66]:
x = collections.deque()
x.append({"rer": 33})
print(x[0])

{'rer': 33}


In [188]:
def get_positions_fixed_strategy(df_merged, block_amount, block_size, block_spacing, shift_theta):

    pos = BlockPosition(df_merged.iloc[0]['tick'], block_size, block_spacing, block_amount)
    pos_array = []

    for index, row in df_merged.iterrows():

        curr_tick = row.tick
        # if(put_in_price_tick != 0):
        #     print(hourly_fee * put_in_price_tick/tvl_per_tick)
        optimized = False
        changed = False

        if(abs(pos.center_tick - curr_tick) > pos.block_amount * pos.block_spacing):
            pos.center_tick = curr_tick
            changed = True
            pos.block_array = pos.gen_block_array()
            print('shift')
        else:
            while(True):
                if(abs(pos.center_tick - curr_tick) >= shift_theta):
                    changed = True
                    if(pos.center_tick > curr_tick):
                        pos.shift_left()
                    else:
                        pos.shift_right()
                else:
                    break

        array_item = {}
        for idx, block in enumerate(pos.block_array):
            array_item['index_'+str(idx)] = block['index']
            array_item['lower_tick_'+str(idx)] = block['lower_tick']
            array_item['upper_tick_'+str(idx)] = block['upper_tick']
        array_item['periodStartUnix'] = row['periodStartUnix']
        pos_array.append(array_item)

    pos_df = pd.DataFrame(pos_array)

    return pos_df

# STEP 3 (TODO): Generate df_positions
df_positions = get_positions_fixed_strategy(df_merged, 5, 180, 180, 120)
df_positions

shift
shift
shift
shift


,index_0,lower_tick_0,upper_tick_0,index_1,lower_tick_1,upper_tick_1,index_2,lower_tick_2,upper_tick_2,index_3,lower_tick_3,upper_tick_3,index_4,lower_tick_4,upper_tick_4,periodStartUnix
0,1,-81684,-81505,2,-81504,-81325,3,-81324,-81145,4,-81144,-80965,5,-80964,-80785,1620158400
1,1,-81684,-81505,2,-81504,-81325,3,-81324,-81145,4,-81144,-80965,5,-80964,-80785,1620162000
2,1,-81684,-81505,2,-81504,-81325,3,-81324,-81145,4,-81144,-80965,5,-80964,-80785,1620165600
3,1,-81684,-81505,2,-81504,-81325,3,-81324,-81145,4,-81144,-80965,5,-80964,-80785,1620169200
4,3,-81324,-81145,4,-81144,-80965,5,-80964,-80785,6,-80784,-80605,7,-80604,-80425,1620172800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2135,863,-79095,-78916,862,-78915,-78736,853,-78735,-78556,852,-78555,-78376,859,-78375,-78196,1627894800
2136,863,-79095,-78916,862,-78915,-78736,853,-78735,-78556,852,-78555,-78376,859,-78375,-78196,1627898400
2137,863,-79095,-78916,862,-78915,-78736,853,-78735,-78556,852,-78555,-78376,859,-78375,-78196,1627902000
2138,863,-79095,-78916,862,-78915,-78736,853,-78735,-78556,852,-78555,-78376,859,-78375,-78196,1627905600


In [57]:
df_merged['tick']

0      -81234
1      -81234
2      -81234
3      -81234
4      -80932
        ...  
2135   -78612
2136   -78612
2137   -78592
2138   -78574
2139   -78581
Name: tick, Length: 2140, dtype: int64

In [42]:
df_merged['tick']

0      -81234
1      -81234
2      -81234
3      -81234
4      -80932
        ...  
2135   -78612
2136   -78612
2137   -78592
2138   -78574
2139   -78581
Name: tick, Length: 2140, dtype: int64

In [92]:
# L_you calculation / inRange
# 4B. JNP: Use 'get_liquidity' function to calculate liquidity as a function of amounts and price range
def get_liquidity0(sqrtA,sqrtB,amount0,decimals):
    
    if (sqrtA > sqrtB):
          (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
    liquidity=amount0/((2**96*(sqrtB-sqrtA)/sqrtB/sqrtA)/10**decimals)
    return liquidity

def get_liquidity1(sqrtA,sqrtB,amount1,decimals):
    
    if (sqrtA > sqrtB):
        (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
    liquidity=amount1/((sqrtB-sqrtA)/2**96/10**decimals)
    return liquidity

def get_amounts(tick,tickA,tickB,targetAmount,decimal0,decimal1):
    
        sqrt=(1.0001**(tick/2)*(2**96))
        sqrtA=(1.0001**(tickA/2)*(2**96))
        sqrtB=(1.0001**(tickB/2)*(2**96))
        if (sqrtA > sqrtB):
            (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
        if sqrt<=sqrtA:
            inRange = False
            return (targetAmount, 0), inRange
        elif sqrt<sqrtB and sqrt>sqrtA:

            liquidity0=get_liquidity0(sqrt,sqrtB,amount0,decimal0)
            liquidity1=get_liquidity1(sqrtA,sqrt,amount1,decimal1)
            
            liquidity=liquidity0 if liquidity0<liquidity1 else liquidity1
            inRange = True
            return liquidity, inRange
        
        else:
            inRange = False
            return (0, targetAmount), inRange


In [190]:
def convert_token0_to_1(tick, token1):
    P = 1.0001**tick
    return token1*P

print(convert_token0_to_1(-81241, 1000))
    
def get_amounts(targetAmount, tick, tick_lower, tick_upper):
    Pl = 1.0001**tick_lower
    P = 1.0001**tick
    Pu = 1.0001**tick_upper
    
    if P >= Pu:
        return (0, targetAmount)
    elif Pl <= P and P < Pu:
        deltaL = targetAmount / ((sqrt(P) - sqrt(Pl)) + (1 / sqrt(P) - 1 / sqrt(Pu)) * P)
        y = deltaL * (1 / sqrt(P) - 1 / sqrt(Pu))
        x = deltaL * (sqrt(P) - sqrt(Pl))
        return (y, x)
    elif Pl > P:
        return (targetAmount/P, 0)

get_amounts(convert_token0_to_1(-81244, 1000), -81237, -81242, -81238)


0.2964316697984161


(0, 0.2963427580804129)

In [187]:
df_positions

,index_0,lower_tick_0,upper_tick_0,index_1,lower_tick_1,upper_tick_1,index_2,lower_tick_2,upper_tick_2,index_3,lower_tick_3,upper_tick_3,index_4,lower_tick_4,upper_tick_4
0,1,-81684,-81505,2,-81504,-81325,3,-81324,-81145,4,-81144,-80965,5,-80964,-80785
1,1,-81684,-81505,2,-81504,-81325,3,-81324,-81145,4,-81144,-80965,5,-80964,-80785
2,1,-81684,-81505,2,-81504,-81325,3,-81324,-81145,4,-81144,-80965,5,-80964,-80785
3,1,-81684,-81505,2,-81504,-81325,3,-81324,-81145,4,-81144,-80965,5,-80964,-80785
4,3,-81324,-81145,4,-81144,-80965,5,-80964,-80785,6,-80784,-80605,7,-80604,-80425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2135,863,-79095,-78916,862,-78915,-78736,853,-78735,-78556,852,-78555,-78376,859,-78375,-78196
2136,863,-79095,-78916,862,-78915,-78736,853,-78735,-78556,852,-78555,-78376,859,-78375,-78196
2137,863,-79095,-78916,862,-78915,-78736,853,-78735,-78556,852,-78555,-78376,859,-78375,-78196
2138,863,-79095,-78916,862,-78915,-78736,853,-78735,-78556,852,-78555,-78376,859,-78375,-78196


In [210]:
def calculate_pnl_from_positions(df_merged, df_positions):

    # TODO: re-calculate amount0 and amount1 to account for every swaps, not just deltas of that hour
    # use amount0_new and amount1_new instead of amountUSD, and get total accured fees in tokens
    # Swap_amountIn > so the fees is get from the (+) positive tokens
    # Therefore, fees you get is in 
    # Convert those tokens into value (convert using TODAY's value or mints's value?)
    
    
    merge_feat = ['periodStartUnix', 'tick','liquidity', 'amountUSD', 'amount0_p', 'amount1_p']
#     merge_feat = ['periodStartUnix', 'tick','liquidity', 'amountUSD', 'amount0_n', 'amount1_n']

    df_positions_fees = df_positions.merge(df_merged[merge_feat], how='left', on='periodStartUnix')
    df_positions_fees['liquidity'] = df_positions_fees['liquidity'].astype(float)

    df_positions_fees = df_positions_fees.reset_index() # cleaning it just in case
    for index, row in df_positions_fees.iterrows():
        row = row.copy()
        
        amount0, amount1 = get_amounts(convert_token0_to_1(row['tick'], 10000), row['tick'], row['lower_tick_2'], row['upper_tick_2'])
            
        print(row['lower_tick_2'], row['tick'], row['upper_tick_2'])
        print(amount0, amount1)

        fees_0, fees_1 = get_fees(row['lower_tick_2'],row['upper_tick_2'],row['tick'],
                        float(row['liquidity']),row['amountUSD'],
                        row['amount0_p'], row['amount1_p'],
                        350000, 100, DECIMAL0, DECIMAL1, FEETIER, row['tick'])
        
       
#         fees_0, fees_1 = get_fees(row['lower_tick'],row['upper_tick'],row['tick'],
#                         float(row['liquidity']),row['amountUSD'],
#                         abs(row['amount0_n']), abs(row['amount1_n']),
#                         MINT_AMT0, MINT_AMT1, DECIMAL0, DECIMAL1, FEETIER, MINT_TICK)
        
        df_positions_fees.loc[index, 'fees_0'] = fees_0
        df_positions_fees.loc[index, 'fees_1'] = fees_1
        
    return df_positions_fees

# STEP 4 (TODO): Calculate Fees, IL and PNL at each hour, and append to df_positions
df_positions_fees = calculate_pnl_from_positions(df_merged, df_positions)

-81324.0 -81234.0 -81145.0
4972.129487426722 1.4914636586487962
-81324.0 -81234.0 -81145.0
4972.129487426722 1.4914636586487962
-81324.0 -81234.0 -81145.0
4972.129487426722 1.4914636586487962
-81324.0 -81234.0 -81145.0
4972.129487426722 1.4914636586487962
-80964.0 -80932.0 -80785.0
8208.069126926894 0.5478540555585228
-80964.0 -80922.0 -80785.0
7649.36691321197 0.7193872669050843
-80964.0 -80922.0 -80785.0
7649.36691321197 0.7193872669050843
-80964.0 -80922.0 -80785.0
7649.36691321197 0.7193872669050843
-80964.0 -80922.0 -80785.0
7649.36691321197 0.7193872669050843
-80964.0 -80922.0 -80785.0
7649.36691321197 0.7193872669050843
-80964.0 -80922.0 -80785.0
7649.36691321197 0.7193872669050843
-80964.0 -80922.0 -80785.0
7649.36691321197 0.7193872669050843
-80964.0 -80922.0 -80785.0
7649.36691321197 0.7193872669050843
-80964.0 -80922.0 -80785.0
7649.36691321197 0.7193872669050843
-80964.0 -80922.0 -80785.0
7649.36691321197 0.7193872669050843
-81324.0 -81323.0 -81145.0
9943.887910437257 0.016

-82489.0 -82467.0 -82310.0
8767.309761241555 0.32324864713168805
-82489.0 -82474.0 -82310.0
9159.149089127677 0.22034224347488923
-82489.0 -82414.0 -82310.0
5808.292240766093 1.1050333905209333
-82669.0 -82626.0 -82490.0
7593.522680850956 0.6210972334700073
-82669.0 -82588.0 -82490.0
5473.80823528343 1.172630021056178
-82489.0 -82459.0 -82310.0
8319.871164849239 0.4409331618570155
-82489.0 -82478.0 -82310.0
9383.208980483743 0.1615634433878851
-82669.0 -82524.0 -82490.0
1903.7113751357388 2.111025046794074
-82489.0 -82452.0 -82310.0
7928.656603476577 0.543984183012907
-82309.0 -82148.0 -82130.0
1008.8220988227496 2.434179782921174
-82309.0 -82205.0 -82130.0
4191.707759233905 1.5635406075314087
-82309.0 -82121.0 -82130.0
0 2.714617201724781
-82129.0 -82010.0 -81950.0
3355.240371465684 1.8239305584028336
-81949.0 -81837.0 -81770.0
3745.649784153015 1.7467234049202385
-81949.0 -81922.0 -81770.0
8487.616250818632 0.4188057416144377
-81589.0 -81558.0 -81410.0
8263.967406796664 0.49855838889

-79635.0 -79642.0 -79456.0
10000.0 0
-79815.0 -79690.0 -79636.0
3020.497464246154 2.4160436493826616
-79635.0 -79530.0 -79456.0
4135.956231502843 2.0626515822770664
-79635.0 -79553.0 -79456.0
5418.064222958664 1.6079733303902541
-79455.0 -79424.0 -79276.0
8263.967406796792 0.61714872635867
-79455.0 -79465.0 -79276.0
10000.0 0
-79635.0 -79555.0 -79456.0
5529.553041107368 1.5685339948586978
-79095.0 -79046.0 -78916.0
7258.545337439798 1.012111923307894
-79275.0 -79218.0 -79096.0
6812.117416743886 1.1568586507356713
-79275.0 -79145.0 -79096.0
2741.4546625602925 2.6533697066219606
-79095.0 -79049.0 -78916.0
7426.015704306929 0.9499988600470886
-79095.0 -79095.0 -78916.0
10000.0 0.0
-79275.0 -79164.0 -79096.0
3801.4134042461014 2.261599644556225
-79275.0 -79125.0 -79096.0
1624.2195046596548 3.0679056940358804
-79455.0 -79346.0 -79276.0
3912.9347144527455 2.1808573022249993
-79455.0 -79418.0 -79276.0
7928.656603476609 0.7367915394916488
-79455.0 -79484.0 -79276.0
10000.0 0
-79635.0 -79632.0 

-79275.0 -79097.0 -79096.0
56.11208956290016 3.6524888783061686
-79275.0 -79214.0 -79096.0
6588.979335816463 1.2383289924850083
-79275.0 -79110.0 -79096.0
784.846660610158 3.380420242248609
-78915.0 -78859.0 -78736.0
6867.909294823454 1.178155743028626
-78915.0 -78718.0 -78736.0
0 3.814974328963822
-78915.0 -78721.0 -78736.0
0 3.813830065525448
-78915.0 -78750.0 -78736.0
784.8466606100918 3.5043261003511272
-78915.0 -78789.0 -78736.0
2964.6959140812837 2.6649628353904538
-78915.0 -78764.0 -78736.0
1568.3045074428783 3.201908219159079
-78915.0 -78724.0 -78736.0
0 3.812686145297462
-78915.0 -78818.0 -78736.0
4581.935777041439 2.0464118447834907
-78915.0 -78843.0 -78736.0
5975.551258957721 1.5162457987369777
-78915.0 -78789.0 -78736.0
2964.6959140812837 2.6649628353904538
-78915.0 -78842.0 -78736.0
5919.796743423391 1.537405518868502
-78915.0 -78884.0 -78736.0
8263.967406796814 0.6513892192751586
-79095.0 -79021.0 -78916.0
5864.043768497242 1.5307669930933572
-79095.0 -79006.0 -78916.0
50

-77475.0 -77365.0 -77296.0
3857.1750165256 2.6829764920038843
-77475.0 -77473.0 -77296.0
9887.784142475171 0.04848560423205401
-77475.0 -77461.0 -77296.0
9215.15333939008 0.3395193429065341
-77475.0 -77482.0 -77296.0
10000.0 0
-77835.0 -77769.0 -77656.0
6310.115937226941 1.5478070710845167
-77835.0 -77764.0 -77656.0
6031.307408521968 1.6655922926202036
-78015.0 -77994.0 -77836.0
8823.267068310068 0.4826261795175846
-78015.0 -77982.0 -77836.0
8152.176231014442 0.7587778505576983
-78015.0 -77921.0 -77836.0
4749.1639866347605 2.16935993362243
-78015.0 -77807.0 -77836.0
0 4.178821968268856
-77835.0 -77789.0 -77656.0
7426.015704306925 1.0775599786438954
-78015.0 -77876.0 -77836.0
2238.931198293916 3.2209124530143014
-78015.0 -77894.0 -77836.0
3243.671424590924 2.7988940454616382
-78015.0 -77893.0 -77836.0
3187.8825832559623 2.8222874748277547
-78015.0 -77874.0 -77836.0
2127.210784991836 3.267930904509091
-78015.0 -77837.0 -77836.0
56.112089562666156 4.142926905732982
-77835.0 -77755.0 -7765

-76215.0 -76079.0 -76036.0
2406.4773191490563 3.7717204633450776
-75855.0 -75722.0 -75676.0
2573.9842956930365 3.8225713875708878
-75855.0 -75848.0 -75676.0
9607.38679005098 0.19956887649333377
-75855.0 -75764.0 -75676.0
4916.388415606905 2.6058424611902424
-76035.0 -76030.0 -75856.0
9719.521814643029 0.13999845054559915
-76035.0 -76063.0 -75856.0
10000.0 0
-76035.0 -75904.0 -75856.0
2685.6351182979356 3.6971970565745838
-75675.0 -75623.0 -75496.0
7091.109047456402 1.51226006155923
-75675.0 -75580.0 -75496.0
4693.421794642669 2.7706456385700067
-75855.0 -75803.0 -75676.0
7091.109047456368 1.4852842399213648
-75855.0 -75691.0 -75676.0
840.8509108722932 4.729346294951428
-75855.0 -75729.0 -75676.0
2964.6959140812905 3.618917011905029
-75495.0 -75344.0 -75316.0
1568.3045074428085 4.507442194433751
-75495.0 -75318.0 -75316.0
112.21585752504011 5.299603010665198
-75495.0 -75447.0 -75316.0
7314.36488170205 1.4209842025999437
-75675.0 -75532.0 -75496.0
2015.4710947825217 4.188902206008908
-75

-76395.0 -76312.0 -76216.0
5362.320910769627 2.250496499593958
-76395.0 -76199.0 -76216.0
0 4.9077782357080375
-76035.0 -75966.0 -75856.0
6142.824983474412 1.9376388383764147
-76035.0 -76011.0 -75856.0
8655.414035545638 0.6724156796360666
-76215.0 -76069.0 -76036.0
1847.8237689856182 4.053256015025603
-76215.0 -76047.0 -76036.0
616.7910195163624 4.675599089147507
-76215.0 -76215.0 -76036.0
10000.0 0.0
-76575.0 -76536.0 -76396.0
7816.926645759849 1.0359010347468165
-76575.0 -76529.0 -76396.0
7426.015704306937 1.222249369349212
-76575.0 -76418.0 -76396.0
1232.6902387584844 4.209598807963831
-76575.0 -76439.0 -76396.0
2406.4773191491054 3.638360083303173
-76575.0 -76599.0 -76396.0
10000.0 0
-76755.0 -76655.0 -76576.0
4414.701266202099 2.618957788849741
-76755.0 -76628.0 -76576.0
2908.8909525435165 3.3340245436898712
-76575.0 -76469.0 -76396.0
4080.203256576695 2.8279150619737203
-76575.0 -76401.0 -76396.0
280.47818535698366 4.674740624240853
-76755.0 -76633.0 -76576.0
3187.882583256092 3.

-77295.0 -77130.0 -77116.0
784.8466606102414 4.120564359544682
-77115.0 -77075.0 -76936.0
7761.068801706076 1.0066613252606973
-77115.0 -77055.0 -76936.0
6644.759628534239 1.5115928190196641
-76935.0 -76883.0 -76756.0
7091.109047456402 1.3332393212895888
-76935.0 -76868.0 -76756.0
6254.350215847258 1.7193300384175163
-76935.0 -76866.0 -76756.0
6142.824983474449 1.770876528580152
-76935.0 -76838.0 -76756.0
4581.9357770416 2.494474387289688
-76935.0 -76790.0 -76756.0
1903.7113751359589 3.7454619691218483
-76935.0 -76799.0 -76756.0
2406.47731914919 3.509715055615109
-76755.0 -76672.0 -76576.0
5362.320910769597 2.1709235112493923
-76755.0 -76723.0 -76576.0
8208.069126926886 0.8345460179399444
-76935.0 -76823.0 -76756.0
3745.649784152943 2.883822365833003
-76935.0 -76772.0 -76756.0
896.8481708151173 4.218838367566756
-76935.0 -76786.0 -76756.0
1680.128835150706 3.850434323011144
-76935.0 -76734.0 -76756.0
0 4.65212459034391
-76935.0 -76767.0 -76756.0
616.7910195163739 4.350805070704466
-767

-75495.0 -75373.0 -75316.0
3187.88258325596 3.6310981321171583
-75495.0 -75458.0 -75316.0
7928.656603476674 1.0947542670579251
-75495.0 -75346.0 -75316.0
1680.1288351506776 4.446773395814995
-75495.0 -75311.0 -75316.0
0 5.363500830376497
-75495.0 -75340.0 -75316.0
1344.585964454445 4.628889447077458
-75495.0 -75367.0 -75316.0
2853.082485929493 3.811844663805334
-75495.0 -75405.0 -75316.0
4972.129487426721 2.671469821740329
-75495.0 -75415.0 -75316.0
5529.5530411074005 2.3729187162309393
-75495.0 -75385.0 -75316.0
3857.17501652564 3.2704150721490066
-75495.0 -75486.0 -75316.0
9495.282765563807 0.2660092345329373
-75495.0 -75425.0 -75316.0
6087.0652855473145 2.0749146860122365
-75315.0 -75275.0 -75136.0
7761.068801706061 1.2051815714254117
-75315.0 -75288.0 -75136.0
8487.616250818535 0.8130348875922083
-75315.0 -75333.0 -75136.0
10000.0 0
-75495.0 -75390.0 -75316.0
4135.956231502776 3.120432557210614
-75495.0 -75418.0 -75316.0
5696.793153275626 2.2834623528296345
-75495.0 -75390.0 -75316

-77655.0 -77547.0 -77476.0
3968.692591477915 2.586761721119275
-77655.0 -77582.0 -77476.0
5919.796743423258 1.743840679973959
-77655.0 -77635.0 -77476.0
8879.230797588045 0.4764743752044536
-77655.0 -77536.0 -77476.0
3355.24037146582 2.853001081675583
-77655.0 -77523.0 -77476.0
2629.811693944392 3.1685881951071306
-77655.0 -77581.0 -77476.0
5864.0437684972785 1.767845746491252
-77655.0 -77615.0 -77476.0
7761.068801706117 0.9537458348642399
-77655.0 -77596.0 -77476.0
6700.542677204682 1.4081844357138087
-77655.0 -77616.0 -77476.0
7816.926645759848 0.9298583768465524
-77655.0 -77530.0 -77476.0
3020.4974642461543 2.9985251389306393
-77655.0 -77546.0 -77476.0
3912.934714452621 2.6109367317448897
-77655.0 -77579.0 -77476.0
5752.542066824813 1.8158684147902964
-77835.0 -77700.0 -77656.0
2462.3172021584996 3.183746241009848
-77835.0 -77661.0 -77656.0
280.4781853567273 4.121346701864084
-77835.0 -77723.0 -77656.0
3745.649784152769 2.6356270524219587
-77835.0 -77708.0 -77656.0
2908.890952543504

In [209]:
TOKEN0_TO_USD = 1
TOKEN1_TO_USD = 2331.7802

fees_token0 = df_positions_fees['fees_0'].sum()*TOKEN0_TO_USD
fees_token1 = df_positions_fees['fees_1'].sum()*TOKEN1_TO_USD
print(fees_token0+fees_token1)

1.559780698198872e-05


In [92]:
# STEP 5 (TODO): also accoutning for minting/burning/swaping costs into PNL

In [93]:
TOKEN0_TO_USD = 1
TOKEN1_TO_USD = 2331.7802

fees_token0 = df_positions_fees['fees_0'].sum()*TOKEN0_TO_USD
fees_token1 = df_positions_fees['fees_1'].sum()*TOKEN1_TO_USD
print(fees_token0+fees_token1)

63893.370392777724


In [94]:
df_positions_fees['fees_0'].sum()

30878.359036860285

In [95]:
df_positions_fees['fees_1'].sum()

14.158715026363739

In [96]:
df_merged[['txCount','swaps_txCount','amountUSD', 'amount0_p', 'amount1_p']].head(19)

,txCount,swaps_txCount,amountUSD,amount0_p,amount1_p
0,21,NaN,NaN,NaN,NaN
1,33,14.0,2.674987e+06,1.853959e+06,355.457852
2,58,31.0,7.960741e+06,4.321993e+06,1574.867926
3,36,24.0,3.867269e+06,3.866513e+06,3.223895
4,15,7.0,1.785534e+05,0.000000e+00,77.966318
5,34,22.0,3.745938e+06,0.000000e+00,1628.510724
6,33,22.0,1.719154e+06,1.522220e+04,736.410654
7,51,36.0,5.017910e+06,5.026098e+06,0.000000
8,29,9.0,8.742947e+05,0.000000e+00,381.005463
9,18,6.0,1.591868e+04,0.000000e+00,6.930991


In [ ]:
df_merged = pd.read_csv('../data/df_merged_DAI-WETH-3000-timestamp-1627466400-1620158400.csv')
watch_list = ['periodStartUnix',  
              'txCount', 'swaps_txCount', # check data integrity
              'amount0', 'amount1', 'amountUSD', # swaps data, note: amountUSD is sqrt(P)
              'tick', 'liquidity', 'sqrtPrice', 'tvlUSD', # pool data at that hour (liquidity is for the whole pool)
              'pool.token0.symbol', 'pool.token1.symbol', # token data
              'token0Price', 'token1Price'
             ]
# https://github.com/Uniswap/uniswap-v3-subgraph/blob/main/schema.graphql
df_merged[watch_list].sample(15)